In [16]:
pip install selenium


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
pip install pandas 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import pandas as pd

In [29]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

In [30]:
driver = webdriver.Chrome()
driver.get('https://www.redbus.in/')
driver.maximize_window()
time.sleep(5)

In [35]:
from urllib.parse import urljoin

In [47]:

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def get_routes_and_links(driver, wait, path, base_url):
    links = []
    routes = []
    
    paths = driver.find_elements(By.XPATH, path)
    
    if not paths:
        print("No elements found on the page.")
        return routes, links
    
    for link in paths:
        href = link.get_attribute("href")
        if href:
            full_url = urljoin(base_url, href)  # Ensure full URL
            links.append(full_url)
        routes.append(link.text)
        
    return routes, links



In [54]:
def click_next_page(driver, wait, i):
    try:
        pagination = wait.until(ec.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
        next_button = pagination.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()="{i+1}"]')
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        time.sleep(3)
        next_button.click()
        time.sleep(5)  # Wait for the page to load
        return True
    except (NoSuchElementException, TimeoutException):
        print(f"No more pages to paginate at step {i}")
        return False


In [56]:
def scrape_routes(url, path, max_pages=5, wait_time=5):
    driver = initialize_driver()
    driver.get(url)
    time.sleep(wait_time)  # Give extra time for the page to fully load

    wait = WebDriverWait(driver, 20)
    all_routes = []
    all_links = []

    try:
        base_url = driver.current_url  # Capture the base URL of the page

        for i in range(1, max_pages + 1):
            routes, links = get_routes_and_links(driver, wait, path, base_url)
            all_routes.extend(routes)
            all_links.extend(links)
            
            if not click_next_page(driver, wait, i):
                break

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        driver.quit()

    return all_routes, all_links


In [81]:
state_links = [
    "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/jksrtc"]
    # Add more state URLs here if needed

In [89]:
# Andhra Pradesh
andhra_routes, andhra_links = scrape_routes(state_links[0], "//a[@class='route']")
df_andhra = pd.DataFrame({"Route_name": andhra_routes, "Route_link": andhra_links})
print("Andhra Pradesh Routes:")
print(df_andhra)

# kerala
kerala_routes, kerala_links = scrape_routes(state_links[1], "//a[@class='route']")
df_kerala = pd.DataFrame({"Route_name": kerala_routes, "Route_link": kerala_links})
print("kerala Routes:")
print(df_kerala)

# west_bengal 
west_bengal_routes, west_bengal_links = scrape_routes(state_links[2], "//a[@class='route']")
df_west_bengal = pd.DataFrame({"Route_name": west_bengal_routes, "Route_link": west_bengal_links})
print("west_bengal Routes:")
print(df_west_bengal)

#  south_bengal 
south_bengal_routes, south_bengal_links = scrape_routes(state_links[3], "//a[@class='route']")
df_south_bengal = pd.DataFrame({"Route_name": south_bengal_routes, "Route_link": south_bengal_links})
print("south_bengal Routes:")
print(df_south_bengal)
 
# telangana
telangana_routes, telangana_links = scrape_routes(state_links[4], "//a[@class='route']")
df_telangana = pd.DataFrame({"Route_name": telangana_routes, "Route_link": telangana_links})
print("telangana Routes:")
print(df_telangana) 



No more pages to paginate at step 5
Andhra Pradesh Routes:
                                 Route_name  \
0                   Vijayawada to Hyderabad   
1                   Hyderabad to Vijayawada   
2                 Kakinada to Visakhapatnam   
3                 Visakhapatnam to Kakinada   
4    Chittoor (Andhra Pradesh) to Bangalore   
5                       Kadapa to Bangalore   
6   Anantapur (andhra pradesh) to Bangalore   
7                     Tirupati to Bangalore   
8               Visakhapatnam to Vijayawada   
9                       Ongole to Hyderabad   
10                    Bangalore to Tirupati   
11   Macherla (andhra pradesh) to Hyderabad   
12             Rajahmundry to Visakhapatnam   
13                     Nandyal to Hyderabad   
14                      Bangalore to Kadapa   
15                      Hyderabad to Ongole   
16     Guntur (Andhra Pradesh) to Hyderabad   
17              Vijayawada to Visakhapatnam   
18                Rajahmundry to Vijayawada   
1

In [88]:
#punjab
punjab_routes, punjab_links = scrape_routes(state_links[5], "//a[@class='route']")
df_punjab = pd.DataFrame({"Route_name": punjab_routes, "Route_link": punjab_links})
print("punjab Routes:")
print(df_punjab)

#bihar 
bihar_routes, bihar_links = scrape_routes(state_links[6], "//a[@class='route']")
df_bihar = pd.DataFrame({"Route_name": bihar_routes, "Route_link": bihar_links})
print("bihar Routes:")
print(df_bihar)
 
#kadamba
kadamba_routes, kadamba_links = scrape_routes(state_links[7], "//a[@class='route']")
df_kadamba = pd.DataFrame({"Route_name": kadamba_routes, "Route_link": kadamba_links})
print("kadamba Routes:")
print(df_kadamba)

#assam 
assam_routes, assam_links = scrape_routes(state_links[8], "//a[@class='route']")
df_assam = pd.DataFrame({"Route_name": assam_routes, "Route_link": assam_links})
print("assam Routes:")
print(df_assam)

#jammu
jammu_routes, jammu_links = scrape_routes(state_links[9], "//a[@class='route']")
df_jammu = pd.DataFrame({"Route_name": jammu_routes, "Route_link": jammu_links})
print("jammu Routes:")
print(df_jammu)

No more pages to paginate at step 3
punjab Routes:
                    Route_name  \
0             Patiala to Delhi   
1             Delhi to Patiala   
2            Ludhiana to Delhi   
3            Delhi to Ludhiana   
4            Phagwara to Delhi   
5           Jalandhar to Delhi   
6           Delhi to Jalandhar   
7     Patiala to Delhi Airport   
8   Jalandhar to Delhi Airport   
9    Ludhiana to Delhi Airport   
10   Phagwara to Delhi Airport   
11   Delhi Airport to Ludhiana   
12           Delhi to Phagwara   
13           Delhi to Amritsar   
14           Amritsar to Delhi   
15    Delhi Airport to Patiala   
16   Amritsar to Delhi Airport   
17         Kapurthala to Delhi   
18  Delhi Airport to Jalandhar   
19      Chandigarh to Bathinda   
20      Chandigarh to Faridkot   
21       Chandigarh to Patiala   

                                           Route_link  
0   https://www.redbus.in/bus-tickets/patiala-to-d...  
1   https://www.redbus.in/bus-tickets/delhi-to-pat... 

In [140]:
df_state0 = pd.concat([df_andhra,df_kerala,df_west_bengal,df_south_bengal,df_telangana,df_punjab,df_bihar,df_kadamba,df_assam,df_jammu],ignore_index=True)


In [106]:
df_state1 = pd.concat([df_andhra,df_kerala,df_west_bengal,df_south_bengal,df_telangana],ignore_index=True)
df_state1

,Route_name,Route_link
0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
2,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
3,Visakhapatnam to Kakinada,https://www.redbus.in/bus-tickets/visakhapatna...
4,Chittoor (Andhra Pradesh) to Bangalore,https://www.redbus.in/bus-tickets/chittoor-and...
...,...,...
139,Hyderabad to Warangal,https://www.redbus.in/bus-tickets/hyderabad-to...
140,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
141,Hyderabad to Guntur (Andhra Pradesh),https://www.redbus.in/bus-tickets/hyderabad-to...
142,Kadapa to Hyderabad,https://www.redbus.in/bus-tickets/kadapa-to-hy...


In [148]:
df_state2 = pd.concat([df_punjab,df_bihar,df_kadamba,df_assam,df_jammu],ignore_index=True)
df_state3 = pd.concat([df_kadamba,df_assam,df_jammu],ignore_index=True)
df_state3

,Route_name,Route_link
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,Goa to Pune,https://www.redbus.in/bus-tickets/goa-to-pune
2,Mumbai to Goa,https://www.redbus.in/bus-tickets/mumbai-to-goa
3,Goa to Mumbai,https://www.redbus.in/bus-tickets/goa-to-mumbai
4,Pandharpur to Goa,https://www.redbus.in/bus-tickets/pandharpur-t...
...,...,...
98,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...
99,Jammu (j and k) to Kishtwar,https://www.redbus.in/bus-tickets/jammu-to-kis...
100,Jammu (j and k) to Mendhar (J & K),https://www.redbus.in/bus-tickets/jammu-to-men...
101,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...


In [119]:
df_state1.to_csv("df_state1.csv",index=False)
df_state2.to_csv("df_state2.csv",index=False)

In [141]:
route_names = []
route_links = []
bus_names = []
bus_types= []
departing_time = []
duration = []
reaching_time = []
star_rating = []
prices = []
seats_available = []
driver = webdriver.Chrome()
driver.maximize_window()

def right_button():
    try:
        right_button = driver.find_elements(By.XPATH, "//i[@class = 'icon icon-right']")
        right_button[0].click()
        time.sleep(6)
    except:
        pass

for i,r in df_state0.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

    # Loop through each link
    driver.get(link)
    time.sleep(3)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()

    right_button()
    # click elements to views bus
    try:
        clicks = driver.find_elements(By.XPATH, "//div[@class='button']")
        for x in clicks:
            x.click()
            time.sleep(1)
    except:
        continue  
    time.sleep(2)
    
    # Initialize scrolling
    SCROLL_PAUSE_TIME = 1

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    # Extract bus details
    bus_name_drive = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type_drive = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    departing_time_drive = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    duration_drive = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    reaching_time_drive = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    price_drive = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats_drive = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    try:
        star_rating_drive = driver.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue

    # Append data to respective lists
    for bus in bus_name_drive:
        route_links.append(link)

    for bus in bus_name_drive:
        route_names.append(routes)

    for bus in bus_name_drive:
        bus_names.append(bus.text)

    for bus_type_elem in bus_type_drive:
        bus_types.append(bus_type_elem.text)

    for start_time_elem in departing_time_drive:
        departing_time.append(start_time_elem.text)

    for reaching_time_elem in  reaching_time_drive:
        reaching_time.append( reaching_time_elem.text)

    for duration_elem in duration_drive:
        duration.append(duration_elem.text)

    for ratings in star_rating_drive:
        star_rating.append(ratings.text)

    for price_elem in price_drive:
        prices.append(price_elem.text)

    for seats_elem in seats_drive:
        seats_available.append(seats_elem.text)
        
print("done")


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF6169DEEB2+31554]
	(No symbol) [0x00007FF616957EE9]
	(No symbol) [0x00007FF61681872A]
	(No symbol) [0x00007FF6167FFA9C]
	(No symbol) [0x00007FF6167FF960]
	(No symbol) [0x00007FF61681AD51]
	(No symbol) [0x00007FF6168AC079]
	(No symbol) [0x00007FF61688CDD3]
	(No symbol) [0x00007FF61685A33B]
	(No symbol) [0x00007FF61685AED1]
	GetHandleVerifier [0x00007FF616CE8B2D+3217341]
	GetHandleVerifier [0x00007FF616D35AF3+3532675]
	GetHandleVerifier [0x00007FF616D2B0F0+3489152]
	GetHandleVerifier [0x00007FF616A8E786+750614]
	(No symbol) [0x00007FF61696376F]
	(No symbol) [0x00007FF61695EB24]
	(No symbol) [0x00007FF61695ECB2]
	(No symbol) [0x00007FF61694E17F]
	BaseThreadInitThunk [0x00007FFA2A547374+20]
	RtlUserThreadStart [0x00007FFA2C05CC91+33]


In [149]:

driver = webdriver.Chrome()
driver.maximize_window()

def right_button():
    try:
        right_button = driver.find_elements(By.XPATH, "//i[@class = 'icon icon-right']")
        right_button[0].click()
        time.sleep(6)
    except:
        pass

for i,r in df_state3.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

    # Loop through each link
    driver.get(link)
    time.sleep(3)  

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()

    right_button()
    # click elements to views bus
    try:
        clicks = driver.find_elements(By.XPATH, "//div[@class='button']")
        for x in clicks:
            x.click()
            time.sleep(1)
    except:
        continue  
    time.sleep(2)
    
    # Initialize scrolling
    SCROLL_PAUSE_TIME = 1

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    # Extract bus details
    bus_name_drive = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type_drive = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    departing_time_drive = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    duration_drive = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    reaching_time_drive = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    price_drive = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats_drive = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
    try:
        star_rating_drive = driver.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue

    # Append data to respective lists
    for bus in bus_name_drive:
        route_links.append(link)

    for bus in bus_name_drive:
        route_names.append(routes)

    for bus in bus_name_drive:
        bus_names.append(bus.text)

    for bus_type_elem in bus_type_drive:
        bus_types.append(bus_type_elem.text)

    for start_time_elem in departing_time_drive:
        departing_time.append(start_time_elem.text)

    for reaching_time_elem in  reaching_time_drive:
        reaching_time.append( reaching_time_elem.text)

    for duration_elem in duration_drive:
        duration.append(duration_elem.text)

    for ratings in star_rating_drive:
        star_rating.append(ratings.text)

    for price_elem in price_drive:
        prices.append(price_elem.text)

    for seats_elem in seats_drive:
        seats_available.append(seats_elem.text)
        
print("done")


done


In [155]:
redbus_data= {
    'bus_names':bus_names ,'bus_types':bus_types,'departing_time':departing_time,'duration':duration,'reaching_time':reaching_time,'star_rating':star_rating,'prices':prices,'seats_available':seats_available }

In [169]:
redbus_data2= {
'route_names': route_names,
'route_links' : route_links,
'bus_names':bus_names  ,
'bus_types':bus_types,
'departing_time':departing_time,
'duration':duration,
'reaching_time':reaching_time,
'star_rating':star_rating,
'prices':prices,
'seats_available':seats_available}

In [170]:
redbus_all= pd.DataFrame(redbus_data2)


In [171]:
pd.DataFrame(redbus_data2)

,route_names,route_links,bus_names,bus_types,departing_time,duration,reaching_time,star_rating,prices,seats_available
0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,TSRTC - 1758,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",00:15,04h 50m,05:05,3.4,INR 396,10 Seats available
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,TSRTC - 42162,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:10,06h 10m,07:20,4.2\n55,INR 396,28 Seats available
2,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,TSRTC - 30710,RAJDHANI (A.C. Semi Sleeper),01:20,05h 40m,07:00,4.1\n18,INR 480,32 Seats available
3,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,TSRTC - 42362,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:50,06h 10m,08:00,4.3\n22,INR 396,30 Seats available
4,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,TSRTC - 1590,RAJDHANI (A.C. Semi Sleeper),02:00,05h 40m,07:40,2.7\n27,INR 480,32 Seats available
...,...,...,...,...,...,...,...,...,...,...
10462,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...,Royal Golden,A/C Seater / Sleeper (2+2),14:00,06h 00m,20:00,1.8,600,31 Seats available
10463,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,AC Sleeper (2+1),14:30,05h 30m,20:00,2.2,INR 800,34 Seats available
10464,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...,Neeraj travels,A/C Seater / Sleeper (2+1),15:40,04h 57m,20:37,2.5,INR 799,38 Seats available
10465,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),18:50,04h 50m,23:40,2.2,INR 600,38 Seats available


In [172]:
redbus_all.to_csv("state_all.csv")